In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install youtube-dl

     |████████████████████████████████| 1.9MB 5.4MB/s 


In [ ]:
!pip install fake_useragent

In [ ]:
from __future__ import unicode_literals
import youtube_dl
import os
import numpy as np
import pandas as pd
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
import requests
from fake_useragent import FakeUserAgentError, UserAgent
from bs4 import BeautifulSoup

#Extract trailer links from metadata

In [ ]:
DEVELOPER_KEY = 'apply_your_key'
YOUTUBE_API_SERVICE_NAME = 'youtube'
YOUTUBE_API_VERSION = 'v3'

def youtube_search(q, max_results):
  youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION,
    developerKey=DEVELOPER_KEY)
  search_response = youtube.search().list(
    q=q,
    part='id,snippet',
    maxResults=max_results
  ).execute()
  videos_title = []
  videos_id = []
  for search_result in search_response.get('items', []):
    if search_result['id']['kind'] == 'youtube#video':
      videos_title.append(search_result['snippet']['title'])
      videos_id.append(search_result['id']['videoId'])
  #print('Videos:\n','\n'.join(videos),'\n')
  return videos_title,videos_id

def url_txt(meta_path):
  meta = pd.read_csv(meta_path)
  urls = meta['trailer_url']
  print(urls)
  index = urls.isnull().T
  index = index[index == True]
  index = index.index
  failed_meta = meta.loc[index]
  print(failed_meta)
  failed_title = failed_meta['title'].to_list()
  #print(failed_title)
  '''
  for i in failed_title:
      q = i + 'trailer'
      max_results = 1
      try:
        title,v_id = youtube_search(q,max_results)
        print(title,v_id)
      except HttpError as e:
        print('An HTTP error %d occurred:\n%s' % (e.resp.status, e.content))
  '''

In [ ]:
meta_path = '../data/21K_movie_record/tmdb_metadata_2020_1000.csv'
url_txt(meta_path)

0      https://www.youtube.com/watch?v=kGM4uYZzfu0
1      https://www.youtube.com/watch?v=szby7ZHLnkA
2      https://www.youtube.com/watch?v=jKCj3XuPG8M
3      https://www.youtube.com/watch?v=J79D3hny-e8
4      https://www.youtube.com/watch?v=xOsLIiBStEs
                          ...                     
995    https://www.youtube.com/watch?v=WtJkS-oFw3A
996                                            NaN
997                                            NaN
998                                            NaN
999    https://www.youtube.com/watch?v=nwbpMcRSIN4
Name: trailer_url, Length: 1000, dtype: object
         id                            title  ... vote_count vote_average
48   593402                        Tolo Tolo  ...        867          6.2
93   674944                     The Occupant  ...        496          6.3
99   656563                     Rich in Love  ...        453          7.9
106  706503                      Lost Bullet  ...        422          6.7
112  614292           

#Download trailer from YouTube

In [ ]:
def download_main(meta_path,output_dir):
    meta = pd.read_csv(meta_path)
    urls = meta['trailer_url'].dropna()
    trailer_urls = []
    print(len(urls))
    
    DS_Store = os.path.join(output_dir,'.DS_Store')
    if os.path.exists(DS_Store):
        os.remove(DS_Store)
    f = os.listdir(output_dir)
    done = []
    for t in f:
        t = t.replace('.mp4','')
        t = 'https://www.youtube.com/watch?v=' + t
        done.append(t)

    res = list(set(urls) - set(done))
    print(len(res))

    for url in res:
        print(url)
        trailer = download_trailer(url,output_dir)
        if trailer is True:
            trailer_urls.append(url)
    print(len(trailer_urls))
    url_path = meta_path.replace('meta','url')
    url_path = url_path.replace('csv','txt')
    #save_url(trailer_urls,url_path)

def save_url(trailer_urls,url_path):
    with open(url_path,'wt') as f:
        for u in trailer_urls:
            f.write(str(u) + '\n')
    f.close()

def download_trailer(trailer_url,output_dir):
    ydl_opts = {
    'quiet':True,
    'format':'bestvideo[ext=mp4]+bestaudio[ext=m4a]/bestvideo[ext=mp4]/best[ext=mp4]',
    'outtmpl':os.path.join(output_dir,'%(id)s'),
    'continue_dl':True,
    'postprocessors':[{
    'key':'FFmpegVideoConvertor',
    'preferedformat':'mp4'
    }]
    }
    try:
        with youtube_dl.YoutubeDL(ydl_opts) as ydl:
            ydl.download([trailer_url])
            return True
    except Exception as error:
        return False

In [ ]:
meta_path = '../data/21K_movie_record/tmdb_metadata_2020_1000.csv'
output_dir = '../data/13K_trailer/videos'
#demo_download ten trailers
download_main(meta_path,output_dir)

754
754
https://www.youtube.com/watch?v=fxv73Lo1iBE
https://www.youtube.com/watch?v=FVrOVlS5TMU
https://www.youtube.com/watch?v=VympX86OpWs
https://www.youtube.com/watch?v=kY3SuNvqQPw
https://www.youtube.com/watch?v=TbZgLKjrdnA
https://www.youtube.com/watch?v=XWhZDQkq0bw
https://www.youtube.com/watch?v=o5m9WhdWOQU
https://www.youtube.com/watch?v=fklskMu40oQ
https://www.youtube.com/watch?v=n0s0FJfyqGk
https://www.youtube.com/watch?v=aV7H9r38Vnk
10
